In [1]:
#import needed libraries and custom functions
import helper_func
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FloatSlider
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pyspark.sql.functions import desc
import folium

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
def add_markers(m,list_of_lat_and_long_and_ID):
    i = 0
    for x in list_of_lat_and_long_and_ID:
        if i < 100:
            folium.Marker(location=[x[1],x[2]],popup='row id: ' +str(x[0])+', coods: ('+str(x[1])+','+str(x[2])+' )',size = 0.1).add_to(m)
            i+=1
        else:
            break


In [3]:
#load osm files in parquet format
node = sqlContext.read.load("Data/hungary.osm.pbf.node.parquet")
way = sqlContext.read.load("Data/hungary.osm.pbf.way.parquet")
relations = sqlContext.read.load("Data/hungary.osm.pbf.relation.parquet")

In [4]:
node.printSchema()

root
 |-- id: long (nullable = true)
 |-- version: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- changeset: long (nullable = true)
 |-- uid: integer (nullable = true)
 |-- user_sid: binary (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [5]:
#This function filters the rows that have tags that contain the word highway
f1 = helper_func.filter_string_match('highway')
filtered_way = way.select('*').filter(f1(way.tags))

#This function creates a new row with the value of tags having key 'highway'
f2 = helper_func.make_column_tag_value_key('highway')
extracted_highway = filtered_way.withColumn('extract_tag',f2(filtered_way.tags))

In [6]:
#Let's just see the filtered value and the row_id
only_tags = extracted_highway.select('id','extract_tag')

In [7]:
only_tags.show()

+-------+-----------+
|     id|extract_tag|
+-------+-----------+
|3175810|residential|
|3175943|residential|
|3175983|residential|
|3192356|  secondary|
|3212111|  secondary|
|3390680|    primary|
|3414164|   tertiary|
|3425635|  secondary|
|3426019|  secondary|
|3426020|residential|
|3426236|  secondary|
|3427029|  secondary|
|3427031|  secondary|
|3427032|  secondary|
|3427034|  secondary|
|3427036|  secondary|
|3427783|    primary|
|3427999|residential|
|3428708|residential|
|3428709|   tertiary|
+-------+-----------+
only showing top 20 rows



In [8]:
#This is the descending order of the different highways in Hungary 
only_tags.groupBy("extract_tag").count().sort(desc("count")).show()


+-------------+------+
|  extract_tag| count|
+-------------+------+
|        track|240048|
|  residential|167600|
|      service|108138|
|      footway| 63573|
|         path| 43916|
|    secondary| 26914|
|      primary| 21218|
| unclassified| 19533|
|     tertiary| 18214|
|        steps|  8554|
|     cycleway|  6585|
|     motorway|  3416|
|living_street|  3261|
|   pedestrian|  2762|
|motorway_link|  2175|
|     platform|  1593|
|        trunk|  1240|
| primary_link|  1179|
| construction|   984|
|   trunk_link|   881|
+-------------+------+
only showing top 20 rows



In [9]:
#SIMILAR To the previosu function, we extract the rows with taxi
f1 = helper_func.filter_string_match('taxi')
filtered_node = node.select('id','latitude','longitude','tags').filter(f1(node.tags))

In [10]:
taxi_df = filtered_node.collect()

In [11]:
#extract the coordinates of each taxi station
taxi_coods = helper_func.get_lat_long(taxi_df)

In [12]:
# We use folium to visualize the taxi stations
m = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)

In [13]:
add_markers(m,taxi_coods)
m

# VISUALISATION OF DIFFERENT KINDS OF AMENITITES

In [14]:

#extract rows with tag that contain amenity
f1 = helper_func.filter_string_match('amenity')
filtered_node = node.select('id','latitude','longitude','tags').filter(f1(node.tags))

#make new column with the value of the key 'amenity'
f2 = helper_func.make_column_tag_value_key('amenity')
extracted_amenities = filtered_node.withColumn('extract_tag',f2(filtered_node.tags))
#make new column with the value of the key 'name'
f3 = helper_func.make_column_tag_value_key('name')
extracted_amenities = extracted_amenities.withColumn('extract_name',f3(extracted_amenities.tags))

In [15]:
amenities_df = extracted_amenities.collect()

In [16]:
# extracted_amenities.groupBy("extract_tag").count().sort(desc("count")).show()
extracted_amenities.printSchema()

root
 |-- id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- extract_tag: string (nullable = true)
 |-- extract_name: string (nullable = true)



In [17]:
extracted_amenities.groupBy("extract_tag").count().sort(desc("count")).show()

+----------------+-----+
|     extract_tag|count|
+----------------+-----+
|  drinking_water|10661|
|   hunting_stand|10576|
|           bench|10184|
|             pub| 5894|
|    waste_basket| 5033|
|      restaurant| 4657|
|       recycling| 3156|
| bicycle_parking| 3027|
|       fast_food| 2624|
|         parking| 2307|
|            cafe| 1993|
|       telephone| 1895|
|         shelter| 1664|
|        pharmacy| 1616|
|place_of_worship| 1601|
|             atm| 1453|
|            bank| 1400|
| vending_machine| 1299|
|     post_office| 1283|
|         toilets| 1280|
+----------------+-----+
only showing top 20 rows



In [18]:
#function for extracting a specific type of amenity
def filter_amenitites(amenities_df,string):
    list1 =[]
    for x in amenities_df:
        if x.extract_tag == string:
            list1.append(x)
    return list1
        

In [19]:
#function for adding points on the map
def add_markers2(m,list_of_lat_and_long_and_ID):
    i = 0
    for x in list_of_lat_and_long_and_ID:
        if i < 100:
            folium.Marker(location=[x[1],x[2]],popup=str(x[3]),size = 0.1).add_to(m)
            i+=1
        else:
            break

In [20]:
#create map
m1 = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)
#function for extracting coods from Dataframe
def get_lat_long(df):
    list1 = []
    for x in df:
        list1.append([x.id,x.latitude , x.longitude,x.extract_name]) 
    return list1     

In [21]:
@interact
def show_articles_more_than(column='drinking_water'):
    filtered =  filter_amenitites(amenities_df,column)
    filtered_coods = get_lat_long(filtered)
    m1 = folium.Map(world_copy_jump=False,location=[47.5145367, 19.0385427],no_wrap=False,zoom_start=7)
    add_markers2(m1,filtered_coods[:100])
    return m1


# YOU CAN CHANGE THE AMENITY NAME HERE AND SEE WHERE IT LIES ON THE MAP
# WHEN YOU CLICK ON ONE OF THE POINTS THE NAME POPS

interactive(children=(Text(value='drinking_water', description='column'), Output()), _dom_classes=('widget-int…

In [22]:
@interact
def show_articles_more_than(column='fuel'):
    filtered =  filter_amenitites(amenities_df,column)
    filtered_coods = get_lat_long(filtered)
    m1 = folium.Map(world_copy_jump=False,location=[47.5145367, 19.0385427],no_wrap=False,zoom_start=7)
    add_markers2(m1,filtered_coods[:2000])
    return m1


# YOU CAN CHANGE THE AMENITY NAME HERE AND SEE WHERE IT LIES ON THE MAP

interactive(children=(Text(value='fuel', description='column'), Output()), _dom_classes=('widget-interact',))